<a href="https://colab.research.google.com/github/kurtsenol/machine-learning/blob/master/TopicModeling/Methodology/prodigy_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !python -m spacy download en_core_web_md

In [ ]:
!pip3 install pickle5

     |████████████████████████████████| 256 kB 5.1 MB/s 


In [ ]:
import pickle5 as pickle

In [ ]:
import os
import numpy as np
import pandas as pd

import re
import random
import unicodedata
import string
import json

In [ ]:
!pip install Pattern
from pattern.text.en import singularize

     |████████████████████████████████| 22.2 MB 21.2 MB/s 
     |████████████████████████████████| 87 kB 2.5 MB/s 
     |████████████████████████████████| 81 kB 300 kB/s 
     |████████████████████████████████| 5.6 MB 22.7 MB/s 
     |████████████████████████████████| 5.6 MB 9.6 MB/s 
     |████████████████████████████████| 419 kB 35.6 MB/s 
     |████████████████████████████████| 104 kB 43.6 MB/s 
     |████████████████████████████████| 3.6 MB 41.9 MB/s 
  Created wheel for Pattern: filename=Pattern-3.6-py3-none-any.whl size=22332721 sha256=2cca2a189cc26e5d2bc960dc8b926e0f77e0417d94d29fde24ec53bea6839e13
  Stored in directory: /root/.cache/pip/wheels/8d/1f/4e/9b67afd2430d55dee90bd57618dd7d899f1323e5852c465682
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99977 sha256=d39fda1feadefb28f51360bedd2aac0b77af63fc021af0055bb751aee76636c0
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
  

In [ ]:
# spaCy
import spacy
# nlp = spacy.load('en_core_web_md')
nlp = spacy.load('en_core_web_sm')

In [ ]:
# nltk
import nltk
nltk.download(['stopwords', 'punkt'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/data/84k_only_abstracts.csv", header=None, sep='\n', skiprows=2)
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/data/84k_only_abstracts.zip", header=None, sep='\n')
df

,0
0,"Nowadays, ICT is used by Destination Managemen..."
1,Research on sustainability in tourism has incr...
2,This study focuses on hospitality education an...
3,The article presents two good practices in cul...
4,The history of São Tomé and Príncipe (STP) sho...
...,...
84857,According to the U. S. Environmental Protectio...
84858,The relationship between museums and sustainab...
84859,"In China, the topics of pandemic and economic ..."
84860,The growth in demand for air transport as well...


In [ ]:
df[0].iloc[0]

'Nowadays, ICT is used by Destination Management Organisations, not only to provide tourist information but also to co-create tourism experiences. The main purpose of the paper is to discuss the approach of Polish Destination Management Organisations co-creation of tourism experience with the use of ICT on the example of Małopolska Voivodeship, Poland. The approach of Destination Management Organisations to digital tourism marketing should be based on the awareness of both the potential of ICT-based tools and the context they operate in. ICT-related solutions implemented by Destination Management Organisations have to be matched to the ICT infrastructure that exists at the destination or region and the needs of tourists who visit the area. The co-creation of tourism experience is beneficial both for Destination Management Organisations and tourists, but this approach should only be applied when both sides are ready to cooperate. ;;'

In [ ]:
df[0] = df[0].str.strip(';')
df[0] = df[0].str.strip()

In [ ]:
df.columns= ["Abstract"]
df

,Abstract
0,"Nowadays, ICT is used by Destination Managemen..."
1,Research on sustainability in tourism has incr...
2,This study focuses on hospitality education an...
3,The article presents two good practices in cul...
4,The history of São Tomé and Príncipe (STP) sho...
...,...
84857,According to the U. S. Environmental Protectio...
84858,The relationship between museums and sustainab...
84859,"In China, the topics of pandemic and economic ..."
84860,The growth in demand for air transport as well...


In [ ]:
df[df.Abstract.str.contains('© 2')]

,Abstract
84861,The growth in demand for air transport as well...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84862 entries, 0 to 84861
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Abstract  84862 non-null  object
dtypes: object(1)
memory usage: 663.1+ KB


In [ ]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [ ]:
def clean(s):
  printable = set(string.printable)
  return ''.join(filter(lambda x: x in printable, s))

In [ ]:
clean("Research on sustainability in tourism has incr..")

'Research on sustainability in tourism has incr..'

In [ ]:
df.Abstract.iloc[0][320:330]

'f Małopols'

In [ ]:
clean(df.Abstract.iloc[0][320:330])

'f Maopols'

In [ ]:
df.Abstract = df.Abstract.apply(clean)

In [ ]:
df.Abstract

0        Nowadays, ICT is used by Destination Managemen...
1        Research on sustainability in tourism has incr...
2        This study focuses on hospitality education an...
3        The article presents two good practices in cul...
4        The history of So Tom and Prncipe (STP) shows ...
                               ...                        
84857    According to the U. S. Environmental Protectio...
84858    The relationship between museums and sustainab...
84859    In China, the topics of pandemic and economic ...
84860    The growth in demand for air transport as well...
84861    The growth in demand for air transport as well...
Name: Abstract, Length: 84862, dtype: object

In [ ]:
df.Abstract.unique()

array(['Nowadays, ICT is used by Destination Management Organisations, not only to provide tourist information but also to co-create tourism experiences. The main purpose of the paper is to discuss the approach of Polish Destination Management Organisations co-creation of tourism experience with the use of ICT on the example of Maopolska Voivodeship, Poland. The approach of Destination Management Organisations to digital tourism marketing should be based on the awareness of both the potential of ICT-based tools and the context they operate in. ICT-related solutions implemented by Destination Management Organisations have to be matched to the ICT infrastructure that exists at the destination or region and the needs of tourists who visit the area. The co-creation of tourism experience is beneficial both for Destination Management Organisations and tourists, but this approach should only be applied when both sides are ready to cooperate.',
       'Research on sustainability in tourism has

In [ ]:
data_list = df.Abstract.unique().tolist()

In [ ]:
sentences = []
nlp = spacy.load("en_core_web_sm")
for item in data_list:
  doc = nlp(item)
  for sent in doc.sents:
    sentences.append(sent.text)

In [ ]:
strip_chars = string.punctuation + " "
strip_chars

In [ ]:
sentences = [s.lstrip(strip_chars) for s in sentences if len(s.split())>3] 
sentences = [s.strip() for s in sentences] 
sentences

In [ ]:
# with open('/content/drive/My Drive/Data/sentences.pkl', 'wb') as file:
#      pickle.dump(sentences,file)

In [ ]:
with open("/content/drive/My Drive/Data/sentences.pkl", "rb") as fp:   # Unpickling
  sents = pickle.load(fp)

In [ ]:
sents_list = [{"text": s} for s in sents]
sents_list[:5]

[{'text': 'Nowadays, ICT is used by Destination Management Organisations, not only to provide tourist information but also to co-create tourism experiences.'},
 {'text': 'The main purpose of the paper is to discuss the approach of Polish Destination Management Organisations co-creation of tourism experience with the use of ICT on the example of Maopolska Voivodeship, Poland.'},
 {'text': 'The approach of Destination Management Organisations to digital tourism marketing should be based on the awareness of both the potential of ICT-based tools and the context they operate in.'},
 {'text': 'ICT-related solutions implemented by Destination Management Organisations have to be matched to the ICT infrastructure that exists at the destination or region and the needs of tourists who visit the area.'},
 {'text': 'The co-creation of tourism experience is beneficial both for Destination Management Organisations and tourists, but this approach should only be applied when both sides are ready to coo

In [ ]:
len(sents)

402923

In [ ]:
with open("/content/drive/My Drive/Data/dataset.jsonl", 'w') as f:
    for item in sents_list:
        f.write(json.dumps(item) + "\n")

In [ ]:
df_taxonomy = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/data/Taxonomy_july_8th.xlsx")
df_taxonomy

,Terminology extraction,Entity
0,Coding,Qualitative data analysis
1,Grounded theory,Qualitative data analysis
2,Computer assisted qualitative data analysis,Qualitative data analysis
3,CAQDAS,Qualitative data analysis
4,Content analysis,Qualitative data analysis
...,...,...
536,Concept mapping,Case study
537,Qualitative comparative analysis,Case study
538,Single cases,Case study
539,Full factorial design,Factorial design


In [ ]:
tax_list = df_taxonomy["Terminology extraction"].unique().tolist()

In [ ]:
# remove "\xa0" from strings and uppercase 
tax_list = [unicodedata.normalize("NFKD",t).upper() for t in tax_list]
tax_list = [t.strip("\u200e") for t in tax_list]

tax_list = [t.upper() if len(t)<6 else t.lower() for t in tax_list]
tax_list = list(set(tax_list))
tax_list

['AMOS',
 'semantic role labelling',
 'pointinterp',
 'decision trees',
 'NNIDW',
 'mobile methodologies',
 'external reliability',
 'forecasting',
 'dbscan',
 'detrending',
 'radial tree',
 'dimensionality reduction',
 'think-aloud',
 'non-probability sampling',
 'random forest',
 'metric functions',
 'IAT',
 'structural  analysis',
 'split-half reliability',
 'IPA',
 'inverse reinforcement learning',
 'geocoding',
 'morphological analysis',
 'geographical analysis',
 'ICA',
 'life story interview',
 'latent semantic analysis',
 'semistructured interview',
 'NMF',
 'archival research',
 'pre-experimental research',
 'sieve diagram',
 'qualitative data analysis',
 'naive bayes',
 'max-diff conjoint analysis',
 'box-and-whisker plot',
 'relationship extraction',
 'kriging',
 'systematic sampling',
 'deep learning',
 'GSR',
 'spatial analysis',
 'quantitative research',
 'biography',
 'classification',
 'longitude',
 'non negative matrix factorization ',
 'full factorial design',
 'accid

In [ ]:
len(tax_list)

528

In [ ]:
phrases = [t for t in tax_list if len(t)>5]
phrases[:5]

['semantic role labelling',
 'pointinterp',
 'decision trees',
 'mobile methodologies',
 'external reliability']

In [ ]:
phr_patterns = [[{"lower": t} for t in word.split()] for word in phrases]
phr_patterns[:5]
#[{"lower": t} for word in phrases] for t in word.split()]

[[{'lower': 'semantic'}, {'lower': 'role'}, {'lower': 'labelling'}],
 [{'lower': 'pointinterp'}],
 [{'lower': 'decision'}, {'lower': 'trees'}],
 [{'lower': 'mobile'}, {'lower': 'methodologies'}],
 [{'lower': 'external'}, {'lower': 'reliability'}]]

In [ ]:
phr_patterns_list = [{"label": "METHOD", "pattern": t} for t in phr_patterns]
phr_patterns_list[:5]

[{'label': 'METHOD',
  'pattern': [{'lower': 'semantic'},
   {'lower': 'role'},
   {'lower': 'labelling'}]},
 {'label': 'METHOD', 'pattern': [{'lower': 'pointinterp'}]},
 {'label': 'METHOD', 'pattern': [{'lower': 'decision'}, {'lower': 'trees'}]},
 {'label': 'METHOD',
  'pattern': [{'lower': 'mobile'}, {'lower': 'methodologies'}]},
 {'label': 'METHOD',
  'pattern': [{'lower': 'external'}, {'lower': 'reliability'}]}]

In [ ]:
acr = [t for t in tax_list if len(t)<6]
acr[:5]

['AMOS', 'NNIDW', 'IAT', 'IPA', 'ICA']

In [ ]:
acr_patterns_list = [{"label": "METHOD", "pattern": t} for t in acr]
acr_patterns_list[:5]

[{'label': 'METHOD', 'pattern': 'AMOS'},
 {'label': 'METHOD', 'pattern': 'NNIDW'},
 {'label': 'METHOD', 'pattern': 'IAT'},
 {'label': 'METHOD', 'pattern': 'IPA'},
 {'label': 'METHOD', 'pattern': 'ICA'}]

In [ ]:
pattern_list = phr_patterns_list + acr_patterns_list
pattern_list[:5]

[{'label': 'METHOD',
  'pattern': [{'lower': 'semantic'},
   {'lower': 'role'},
   {'lower': 'labelling'}]},
 {'label': 'METHOD', 'pattern': [{'lower': 'pointinterp'}]},
 {'label': 'METHOD', 'pattern': [{'lower': 'decision'}, {'lower': 'trees'}]},
 {'label': 'METHOD',
  'pattern': [{'lower': 'mobile'}, {'lower': 'methodologies'}]},
 {'label': 'METHOD',
  'pattern': [{'lower': 'external'}, {'lower': 'reliability'}]}]

In [ ]:
with open("/content/drive/My Drive/Data/method_patterns.jsonl", 'w') as f:
    for item in pattern_list:
        f.write(json.dumps(item) + "\n")

In [ ]:
remove_list = ["BIBLIOMETRICS", 'CODING', "DECISION TREES", 'DISCOURSE ANALYSIS', "ENSEMBLE METHODS", 'DISCRETE', 'EXPERIMENTAL RESEARCH', "EXPLORATORY FACTOR ANALYSIS",'K-NEAREST NEIGHBORS', "NETWORK", 'SAMPLING', 'STATISTICAL', 'SUPPORT VECTOR MACHINES', 'THINK-ALOUD PROTOCOLS',  'TIME-SERIES CROSS-SECTION',
 'TIME-SERIES DATA ANALYSIS', 'TIME-SERIES DATA DESIGN','TELEPHONE SURVEYS', 'THINK ALOUD', 'TOPIC MODELING', 'RELIABILITY', 'VALIDITY', 'STRUCTURED INTERVIEW', 'CHOICE EXPERIMENT']

In [ ]:
tax_list = [t.strip() for t in tax_list if t not in remove_list]

In [ ]:
tax_list = list(set(tax_list))

In [ ]:
sorted(tax_list)

['ABDUCTION',
 'ABDUCTIVE',
 'ACCIDENTAL SAMPLING',
 'ACTION RESEARCH',
 'ADAPTIVE CONJOINT ANALYSIS',
 'ALTMETRICS',
 'AMOS',
 'ANALYTICAL RESEARCH',
 'ANOMALY DETECTION',
 'APPLIED RESEARCH',
 'APRIORI ALGORITHM',
 'ARCHIVAL RESEARCH',
 'AREA CHART',
 'AREA SAMPLING',
 'ARGUMENT MINING',
 'ARIMA',
 'ARL',
 'ASSOCIATIVE REINFORCEMENT LEARNING',
 'ATLAS.TI',
 'AUTOBIOGRAPHY',
 'AUTOENCODERS',
 'AUTOETHNOGRAPHY',
 'AXIAL CODING',
 'BAG OF WORDS',
 'BAR CHART',
 'BAYESIAN STATISTICS',
 'BEHAVIORAL-BASED INTERVIEW',
 'BIBLIOGRAPHIC ANALYSIS',
 'BIBLIOGRAPHICAL COUPLING',
 'BIBLIOMETRIC',
 'BIOGRAPHIC NARRATIVE INTERPRETIVE METHOD',
 'BIOGRAPHY',
 'BIOMETRIC RESEARCH',
 'BNIM',
 'BOOTSTRAPPING',
 'BOW',
 'BOX PLOT',
 'BOX-AND-WHISKER PLOT',
 'BOX-JENKINS MODELING',
 'BRAIN SIGNAL',
 'BUBBLE CLOUD',
 'BULLET GRAPH',
 'CA',
 'CAP ETHNOGRAPHY',
 'CAPI',
 'CAQDAS',
 'CARTOGRAM',
 'CASE STUDY',
 'CASUAL-COMPARATIVE RESEARCH',
 'CDA',
 'CFA',
 'CHARADE',
 'CHOICE-BASED CONJOINT ANALYSIS',
 'CHRO

In [ ]:
len(tax_list)

505

In [ ]:
d = pd.DataFrame({"Entity_List":tax_list})
d

,Entity_List
0,MOSAIC DISPLAY
1,GAUSSIAN PROCESSES
2,NLP
3,EDA
4,ENSEMBLE METHOD
...,...
500,NARRATIVE ANALYSIS
501,DISTRIBUTIONAL SEMANTICS
502,SINTAX
503,INFORMETRICS


In [ ]:
# d.to_excel("/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/data/entity_list_Sep19.xlsx")

In [ ]:
d = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/TopicModeling/Methodology/data/entity_list_Sep19.xlsx",  index_col=0)

In [ ]:
d

,Entity_List
0,TELEPHONE SURVEY
1,DISCOURSE
2,TIME-SERIES CROSS-SECTION
3,SELF ORGANIZING MAPS
4,QUOTA SAMPLING
...,...
500,PIECEWISE CONSTANT INTERPOLATION
501,CORRELATIONAL RESEARCH
502,TEXT AND SPEECH PROCESSING
503,NEURAL NETWORKS


In [ ]:
method_sections_list = df.Abstract.tolist()

In [ ]:
method_sections_list

['Nowadays, ICT is used by Destination Management Organisations, not only to provide tourist information but also to co-create tourism experiences. The main purpose of the paper is to discuss the approach of Polish Destination Management Organisations co-creation of tourism experience with the use of ICT on the example of Maopolska Voivodeship, Poland. The approach of Destination Management Organisations to digital tourism marketing should be based on the awareness of both the potential of ICT-based tools and the context they operate in. ICT-related solutions implemented by Destination Management Organisations have to be matched to the ICT infrastructure that exists at the destination or region and the needs of tourists who visit the area. The co-creation of tourism experience is beneficial both for Destination Management Organisations and tourists, but this approach should only be applied when both sides are ready to cooperate.',
 'Research on sustainability in tourism has increasingl

In [ ]:
len(method_sections_list)

84862

In [ ]:
whole_text = " ".join(method_sections_list)

In [ ]:
print(nltk.tokenize.sent_tokenize(df.Abstract.iloc[0]))

['Nowadays, ICT is used by Destination Management Organisations, not only to provide tourist information but also to co-create tourism experiences.', 'The main purpose of the paper is to discuss the approach of Polish Destination Management Organisations co-creation of tourism experience with the use of ICT on the example of Maopolska Voivodeship, Poland.', 'The approach of Destination Management Organisations to digital tourism marketing should be based on the awareness of both the potential of ICT-based tools and the context they operate in.', 'ICT-related solutions implemented by Destination Management Organisations have to be matched to the ICT infrastructure that exists at the destination or region and the needs of tourists who visit the area.', 'The co-creation of tourism experience is beneficial both for Destination Management Organisations and tourists, but this approach should only be applied when both sides are ready to cooperate.']


In [ ]:
tokenized = nltk.tokenize.sent_tokenize(whole_text)

In [ ]:
# def createTrainData(entity_list, tokenizedDoc, entity_type):
  
#   train_list=[]
#   for entity in entity_list:
    
#     if len(entity)<5:
#       if singularize(entity)!=entity:
#         patern = "|".join([' ' + singularize(entity).upper() + " ", " " + entity + " "])
#         patern = r"{}".format(patern)
#         print(patern)
#       else:
#         patern = " " + entity + " "
#         patern = r"{}".format(patern)
#         print(patern)


#       tokenized_filtered = [part for part in tokenizedDoc if re.search(patern, part)]
#     else: 

#       if singularize(entity)!=entity and "ANALYSIS" not in entity:
#         patern = "|".join([singularize(entity), entity])
#         patern=r"{}".format(patern)
#         print(patern)

#       else:
#         patern=r"{}".format(entity)
#         print(patern)

#       tokenized_filtered = [part for part in tokenizedDoc if re.search(patern, part, re.I)]

#     tokenized_filtered = [part for part in tokenizedDoc if re.search(patern, part, re.I)]

#     if tokenized_filtered:
#       print("here")

In [ ]:
# createTrainData(tax_list, tokenized, "method")

In [ ]:
len(tokenized)

584367

In [ ]:
len(list(set(tokenized)))

391186

In [ ]:
tokenized = list(set(tokenized))

In [ ]:
string.punctuation, type(string.punctuation)

('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~', str)

In [ ]:
"%&sdfs%&sdfsdf%&>?@[\\]^asd>?@[\\]^".strip(string.punctuation)

'sdfs%&sdfsdf%&>?@[\\]^asd'

In [ ]:
strip_chars = string.punctuation + "0123456789 "
strip_chars 

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789 '

In [ ]:
"293%&sdfs%&sdfsdf%&>?@[\\]^asd>?@[\\]^194302<= >?".strip(strip_chars)

'sdfs%&sdfsdf%&>?@[\\]^asd'

In [ ]:
tokenized_1 = [t.strip(strip_chars) for t in tokenized if len(t) > 30] 

In [ ]:
tokenized_1 = [t for t in tokenized_1 if len(t) > 30 if len(t) < 1000] 

In [ ]:
len(tokenized_1)

388434

In [ ]:
tokenized_1 = list(set(tokenized_1))

In [ ]:
len(tokenized_1)

388357

In [ ]:
sorted(tokenized_1[-145000:-143000])

['A Baltic Health Tourism Cluster was established in',
 'A case study of prioritizing different sites (cities) in India for medical tourism services is shown to demonstrate the applicability of the proposed method',
 'A chi-square test was used to determine characteristics between responses',
 'A conceptual model based on PressureStateResponse relationships is put forward to explore the situation in which, in the tourism community, terraces have a greater variety of functions and enhanced values resulting in the spatial expansion of hazard effects',
 'A conceptual model is proposed that allows syndromes to be evaluated',
 'A correlation analysis is employed to understand the relationship between the derived landscape factors and motivations, and then how landscape values relate to them is discussed',
 'A correspondence analysis further exhibits the attitudes toward tourism development between loyal and nonloyal residents',
 'A critique of tourism curriculum proposals in the literature 

In [ ]:
tax_list

['TELEPHONE SURVEY',
 'DISCOURSE',
 'TIME-SERIES CROSS-SECTION',
 'SELF ORGANIZING MAPS',
 'QUOTA SAMPLING',
 'SCATTER PLOT',
 'TREND ANALYSIS',
 'DBNS',
 'POINTINTERP',
 'NARRATIVE INTERVIEWING',
 'SEMI-SUPERVISED LEARNING',
 'SERIAL CORRELATION',
 'COHORT STUDIES',
 'LONG SHORT TERM MEMORY NETWORKS',
 'TREE VIEWER',
 'CONTINUOUS',
 'HISTORICAL RESEARCH',
 'SOCIOMETRY',
 'FACTORIAL DESIGN',
 'CO-WORD ANALYSIS',
 'PAM',
 'DRL',
 'TOKENIZATION',
 'ELECTROENCEPHALOGRAPHY',
 'QUALITATIVE RESEARCH',
 'INTERVENTION ANALYSIS',
 'BUBBLE CLOUD',
 'LATENT BUDGET ANALYSIS',
 'PROGOL',
 'MULTIOUTPUT ALGORITHMS',
 'CONVERGENT VALIDITY',
 'MLPS',
 'NAMED ENTITY RECOGNITION',
 'INVERSE DISTANCE WEIGHTING',
 'SINTAX',
 'COREFERENCE RESOLUTION',
 'HISTORY ANALYSIS',
 'EXPERT INTERVIEWS',
 'PRE-EXPERIMENTAL RESEARCH',
 'GENERATIVE ADVERSARIAL NETWORKS',
 'HDBSCAN',
 'CYBERMETRICS',
 'SURVIVAL ANALYSIS',
 'FULL-PROFILE CONJOINT ANALYSIS',
 'CO-AUTHOR ANALYSIS',
 'MORPHOLOGICAL ANALYSIS',
 'PHENOMENOLOGY

In [ ]:
len(tax_list)

505

In [ ]:
tokenized_1

['Based on the study findings, a balanced model of religious tourism development is proposed, and specific recommendations are offered to sustainably manage religious sites',
 'India, Buddhism, governance, pilgrimage, development, policy, tourism',
 'The learning resources used are in the form of text (pdf), videos and online learning called "kulinera training"; and 3) development phase, which is the compilation of Tengger culinary training materials as a training service from instructures to create and maintain online-based learning',
 'Findings: Tourism organisations analysed consider that digital skills are still an unresolved issue',
 'Finally, we appraise the Tourist-Area-Season topic (TAST) model with the cocktail recommendation approach on the real-world travel package data',
 'Despite the importance of SMMEs for tourism development across sub-Saharan Africa and for pro-poor impacts, research on tourism SMMEs in developing countries (including sub-Saharan Africa) remains embryon

In [ ]:
def addQ(patern):
  if "-" in patern:
    patern = patern.split("-")
    patern = "-?".join(patern)
  return patern

In [ ]:
entity_dict={}
def createTrainData(entity_list, tokenizedDoc, entity_type):
  
  df_temp = pd.DataFrame()

  train_list=[]

  for entity in entity_list:
    
    if len(entity)<5:
      if singularize(entity)!=entity:
        patern = "|".join([' ' + singularize(entity).upper() + " ", " " + entity + " "])
        patern = addQ(patern)
        patern = r"{}".format(patern)
      else:
        patern = " " + entity + " "
        patern = addQ(patern)
        patern = r"{}".format(patern)


      tokenized_filtered = [part for part in tokenizedDoc if re.search(patern, part)]
    else: 

      if singularize(entity)!=entity:
        patern = "|".join([singularize(entity), entity])
        patern = addQ(patern)
        patern=r"{}".format(patern)
      else:
        patern=r"{}".format(entity)
        patern = addQ(patern)
      tokenized_filtered = [part for part in tokenizedDoc if re.search(patern, part, re.I)]


    if tokenized_filtered:
      
      df_train = pd.DataFrame()
      df_train["Description"] = tokenized_filtered
      
      entity_dict[entity] = len(df_train)
      print(entity + ": " + str(len(df_train)))

      # if len(df_train)>50:
      #   df_train=df_train.sample(50, random_state=21)
      
      df_train["Title"] = [[entity]] * len(df_train)

      if len(entity)>4:
        df_train["Location"] = df_train[df_train.Description.notnull()].apply(lambda row: [(m.start(), m.end()) for m in re.finditer(patern, row["Description"], re.I)], axis=1)
      else:
        df_train["Location"] = df_train[df_train.Description.notnull()].apply(lambda row: [(m.start(), m.end()) for m in re.finditer(patern, row["Description"])], axis=1)

      df_temp = pd.concat([df_temp, df_train])

  return df_temp


In [ ]:
# tax_train_list = createTrainData(["ITS"], tokenized[:10], "method")

In [ ]:
# tax_train_list

In [ ]:
df_tax_train = createTrainData(tax_list, tokenized_1, "method")

TELEPHONE SURVEY: 20
DISCOURSE: 1891
QUOTA SAMPLING: 20
SCATTER PLOT: 2
TREND ANALYSIS: 18
SEMI-SUPERVISED LEARNING: 3
SERIAL CORRELATION: 7
COHORT STUDIES: 1
CONTINUOUS: 903
HISTORICAL RESEARCH: 24
FACTORIAL DESIGN: 7
CO-WORD ANALYSIS: 12
PAM: 1
ELECTROENCEPHALOGRAPHY: 4
QUALITATIVE RESEARCH: 496
INTERVENTION ANALYSIS: 6
CONVERGENT VALIDITY: 14
MLPS: 7
NAMED ENTITY RECOGNITION: 8
INVERSE DISTANCE WEIGHTING: 2
COREFERENCE RESOLUTION: 1
HISTORY ANALYSIS: 5
EXPERT INTERVIEWS: 41
HDBSCAN: 4
CYBERMETRICS: 2
SURVIVAL ANALYSIS: 8
MORPHOLOGICAL ANALYSIS: 7
PHENOMENOLOGY: 55
MULTILAYER PERCEPTRONS: 11
GEOCODING: 1
EDA: 2
CROSS-CASE ANALYSIS: 9
GEOGRAPHICAL ANALYSIS: 17
DESCRIPTIVE STATISTICS: 183
GAUSSIAN MIXTURE: 3
K-NEAREST NEIGHBOR: 9
DATA MINING: 195
QUANTITATIVE DATA ANALYSIS: 5
FACTOR ANALYSIS: 703
POLICY RESEARCH: 22
BOOTSTRAPPING: 30
CROSS-NATIONAL RESEARCH: 1
PLSA: 1
COMPUTATIONAL MODELING: 3
UNIVARIATE ANALYSIS: 3
THINK-ALOUD: 2
AREA SAMPLING: 1
INDEPENDENT SAMPLES T-TEST: 6
AMOS: 44

In [ ]:
sorted(entity_dict.items())

[('ABDUCTION', 5),
 ('ABDUCTIVE', 6),
 ('ACCIDENTAL SAMPLING', 24),
 ('ACTION RESEARCH', 114),
 ('ADAPTIVE CONJOINT ANALYSIS', 2),
 ('ALTMETRICS', 5),
 ('AMOS', 44),
 ('ANALYTICAL RESEARCH', 8),
 ('ANOMALY DETECTION', 3),
 ('APPLIED RESEARCH', 69),
 ('APRIORI ALGORITHM', 3),
 ('ARCHIVAL RESEARCH', 36),
 ('AREA SAMPLING', 1),
 ('ARIMA', 196),
 ('ATLAS.TI', 15),
 ('AUTOBIOGRAPHY', 3),
 ('AUTOENCODERS', 3),
 ('AUTOETHNOGRAPHY', 22),
 ('AXIAL CODING', 3),
 ('BAR CHART', 1),
 ('BAYESIAN STATISTICS', 2),
 ('BIBLIOGRAPHIC ANALYSIS', 3),
 ('BIBLIOGRAPHICAL COUPLING', 1),
 ('BIBLIOMETRIC', 232),
 ('BIOGRAPHY', 22),
 ('BOOTSTRAPPING', 30),
 ('CA', 26),
 ('CAQDAS', 1),
 ('CARTOGRAM', 5),
 ('CASE STUDY', 3491),
 ('CDA', 9),
 ('CFA', 18),
 ('CHOICE-BASED CONJOINT ANALYSIS', 2),
 ('CITATION ANALYSIS', 41),
 ('CLASSIFICATION', 928),
 ('CLUSTERING', 369),
 ('CNNS', 4),
 ('CO-CITATION ANALYSIS', 22),
 ('CO-WORD ANALYSIS', 12),
 ('COHORT ANALYSIS', 10),
 ('COHORT STUDIES', 1),
 ('COLLABORATIVE RESEARCH'

In [ ]:
df_tax_train

,Description,Title,Location
0,Data were collected from telephone survey from...,[TELEPHONE SURVEY],"[(25, 41)]"
1,"Based on a telephone survey, the presented stu...",[TELEPHONE SURVEY],"[(11, 27)]"
2,Design/methodology/approach Data were collecte...,[TELEPHONE SURVEY],"[(62, 78)]"
3,In-depth interviews with 12 major stakeholders...,[TELEPHONE SURVEY],"[(51, 67)]"
4,Two face-to-face unstructured in-depth intervi...,[TELEPHONE SURVEY],"[(120, 136)]"
...,...,...,...
0,A series of experimental studies was carried o...,[IMPLICIT ASSOCIATION TEST],"[(73, 98)]"
1,A total of 89 college students were recruited ...,[IMPLICIT ASSOCIATION TEST],"[(83, 108)]"
2,The study examines individuals' attitudes towa...,[IMPLICIT ASSOCIATION TEST],"[(140, 165)]"
3,This study utilized Single Target Implicit Ass...,[IMPLICIT ASSOCIATION TEST],"[(34, 59)]"


In [ ]:
unique_df = df_tax_train[~df_tax_train.duplicated(subset=["Description"], keep=False)]
unique_df 

,Description,Title,Location
0,It explores how these practices can be conside...,[DISCOURSE],"[(82, 91)]"
1,Official Tourism Websites: A Discourse Analysi...,[DISCOURSE],"[(29, 38)]"
2,Based on ethnographic fieldwork among three no...,[DISCOURSE],"[(260, 269)]"
3,I argue that their testimonies of desire are s...,[DISCOURSE],"[(67, 76)]"
4,Drawing on recently developed frameworks for s...,[DISCOURSE],"[(58, 67)]"
...,...,...,...
0,Citation data collection followed a theoretica...,[THEORETICAL SAMPLING],"[(36, 56)]"
1,Design/methodology/approach: Based on theoreti...,[THEORETICAL SAMPLING],"[(38, 58)]"
3,A judgmental theoretical sampling technique ut...,[THEORETICAL SAMPLING],"[(13, 33)]"
3,This study utilized Single Target Implicit Ass...,[IMPLICIT ASSOCIATION TEST],"[(34, 59)]"


In [ ]:
notunique_df = df_tax_train[df_tax_train.duplicated(subset=["Description"], keep=False)].sort_values("Title")
notunique_df 

,Description,Title,Location
3,"Using a critical realism perspective, we ident...",[ABDUCTION],"[(177, 186)]"
1,"With ACE, researchers construct causal histori...",[ABDUCTION],"[(234, 243)]"
1,This study employs the abductive approach with...,[ABDUCTIVE],"[(23, 32)]"
0,The research further demonstrates the value of...,[ABDUCTIVE],"[(47, 56)]"
1,The primary method used in this study is the q...,[ACCIDENTAL SAMPLING],"[(192, 211)]"
...,...,...,...
0,"Based on the review, the study introduces word...",[WORD CLOUD],"[(42, 52)]"
1,We generated a visual trend analysis of the ti...,[WORD CLOUD],"[(147, 157)]"
2,Text mining using word cloud analysis revealed...,[WORD CLOUD],"[(18, 28)]"
0,In spite of decades of research on word sense ...,[WORD SENSE DISAMBIGUATION],"[(35, 60)]"


In [ ]:
df_tax_train[df_tax_train.duplicated(subset=["Description"], keep=False)].groupby("Description", as_index=False)[["Location", "Title"]].sum()

,Description,Location,Title
0,A GIS analysis on the urbanization spread (172...,"[(218, 227), (1, 6)]","[CONTINUOUS, GIS]"
1,A GIS model was developed based on classificat...,"[(1, 6), (35, 49), (114, 128)]","[GIS, CLASSIFICATION]"
2,A Geographic Information System (GIS) has been...,"[(79, 94), (2, 31)]","[SPATIAL ANALYSIS, GEOGRAPHIC INFORMATION SYSTEM]"
3,A Rapid Situation Analysis is used to gain ins...,"[(93, 102), (133, 144)]","[INTERVIEW, OBSERVATION]"
4,A Revealed Preference survey was conducted in ...,"[(22, 28), (87, 96)]","[SURVEY, INTERVIEW]"
...,...,...,...
5914,the development of a web mapping system and it...,"[(222, 233), (25, 32)]","[FORECASTING, MAPPING]"
5915,this paper proposes a new tourism push system ...,"[(116, 130), (102, 130), (59, 72), (116, 130)]","[NEURAL NETWORK, CONVOLUTIONAL NEURAL NETWORKS..."
5916,through field observation and depth interview,"[(36, 45), (14, 25)]","[INTERVIEW, OBSERVATION]"
5917,time difference between web search activities ...,"[(150, 161), (92, 103)]","[FORECASTING, TIME SERIES]"


In [ ]:
df_ex = df_tax_train[df_tax_train.duplicated(subset=["Description"], keep=False)].groupby("Description", as_index=False)[["Location", "Title"]].sum()
df_ex

,Description,Location,Title
0,A GIS analysis on the urbanization spread (172...,"[(218, 227), (1, 6)]","[CONTINUOUS, GIS]"
1,A GIS model was developed based on classificat...,"[(1, 6), (35, 49), (114, 128)]","[GIS, CLASSIFICATION]"
2,A Geographic Information System (GIS) has been...,"[(79, 94), (2, 31)]","[SPATIAL ANALYSIS, GEOGRAPHIC INFORMATION SYSTEM]"
3,A Rapid Situation Analysis is used to gain ins...,"[(93, 102), (133, 144)]","[INTERVIEW, OBSERVATION]"
4,A Revealed Preference survey was conducted in ...,"[(22, 28), (87, 96)]","[SURVEY, INTERVIEW]"
...,...,...,...
5914,the development of a web mapping system and it...,"[(222, 233), (25, 32)]","[FORECASTING, MAPPING]"
5915,this paper proposes a new tourism push system ...,"[(116, 130), (102, 130), (59, 72), (116, 130)]","[NEURAL NETWORK, CONVOLUTIONAL NEURAL NETWORKS..."
5916,through field observation and depth interview,"[(36, 45), (14, 25)]","[INTERVIEW, OBSERVATION]"
5917,time difference between web search activities ...,"[(150, 161), (92, 103)]","[FORECASTING, TIME SERIES]"


In [ ]:
df_ex["Location"] = df_ex["Location"].apply(sorted, key=lambda x: x[0])

In [ ]:
df_ex

,Description,Location,Title
0,A GIS analysis on the urbanization spread (172...,"[(1, 6), (218, 227)]","[CONTINUOUS, GIS]"
1,A GIS model was developed based on classificat...,"[(1, 6), (35, 49), (114, 128)]","[GIS, CLASSIFICATION]"
2,A Geographic Information System (GIS) has been...,"[(2, 31), (79, 94)]","[SPATIAL ANALYSIS, GEOGRAPHIC INFORMATION SYSTEM]"
3,A Rapid Situation Analysis is used to gain ins...,"[(93, 102), (133, 144)]","[INTERVIEW, OBSERVATION]"
4,A Revealed Preference survey was conducted in ...,"[(22, 28), (87, 96)]","[SURVEY, INTERVIEW]"
...,...,...,...
5914,the development of a web mapping system and it...,"[(25, 32), (222, 233)]","[FORECASTING, MAPPING]"
5915,this paper proposes a new tourism push system ...,"[(59, 72), (102, 130), (116, 130), (116, 130)]","[NEURAL NETWORK, CONVOLUTIONAL NEURAL NETWORKS..."
5916,through field observation and depth interview,"[(14, 25), (36, 45)]","[INTERVIEW, OBSERVATION]"
5917,time difference between web search activities ...,"[(92, 103), (150, 161)]","[FORECASTING, TIME SERIES]"


In [ ]:
df_ex[df_ex.Location.apply(len)>3]

In [ ]:
def catchsim(loc):
  for i in range(len(loc)-1):
    if loc[i][1] >= loc[i+1][0]:
      return False
  return True

In [ ]:
catchsim([(75, 85), (188, 197), (218, 241), (242, 248)])

True

In [ ]:
df_ex = df_ex[df_ex.Location.apply(catchsim)]

In [ ]:
sorted(df_ex.Title.explode().unique())

['ABDUCTION',
 'ABDUCTIVE',
 'ACCIDENTAL SAMPLING',
 'ACTION RESEARCH',
 'ALTMETRICS',
 'AMOS',
 'ANALYTICAL RESEARCH',
 'ANOMALY DETECTION',
 'APPLIED RESEARCH',
 'APRIORI ALGORITHM',
 'ARCHIVAL RESEARCH',
 'AREA SAMPLING',
 'ARIMA',
 'ATLAS.TI',
 'AUTOENCODERS',
 'AXIAL CODING',
 'BAR CHART',
 'BAYESIAN STATISTICS',
 'BIBLIOGRAPHIC ANALYSIS',
 'BIBLIOMETRIC',
 'BIOGRAPHY',
 'BOOTSTRAPPING',
 'CA',
 'CAQDAS',
 'CARTOGRAM',
 'CASE STUDY',
 'CDA',
 'CFA',
 'CITATION ANALYSIS',
 'CLASSIFICATION',
 'CLUSTERING',
 'CNNS',
 'CO-WORD ANALYSIS',
 'COHORT ANALYSIS',
 'COLLABORATIVE RESEARCH',
 'CONCEPTUAL ANALYSIS',
 'CONCEPTUAL RESEARCH',
 'CONFIDENCE INTERVALS',
 'CONJOINT',
 'CONSTRUCT VALIDITY',
 'CONTENT ANALYSIS',
 'CONTINUOUS',
 'CONVENIENCE SAMPLING',
 'CONVERGENT VALIDITY',
 'CONVERSATION ANALYSIS',
 'COREFERENCE RESOLUTION',
 'COVERT RESEARCH',
 'CROSS TABULATIONS',
 'CROSS-VALIDATION',
 'DATA MINING',
 'DATA TYPE',
 'DATA VISUALIZATION',
 'DBSCAN',
 'DECISION TREE',
 'DEDUCTION',
 '

In [ ]:
df_f = pd.concat([unique_df, df_ex])

In [ ]:
df_f

,Description,Title,Location
0,It explores how these practices can be conside...,[DISCOURSE],"[(82, 91)]"
1,Official Tourism Websites: A Discourse Analysi...,[DISCOURSE],"[(29, 38)]"
2,Based on ethnographic fieldwork among three no...,[DISCOURSE],"[(260, 269)]"
3,I argue that their testimonies of desire are s...,[DISCOURSE],"[(67, 76)]"
4,Drawing on recently developed frameworks for s...,[DISCOURSE],"[(58, 67)]"
...,...,...,...
5912,survey responses were collected from non-Malac...,"[SURVEY, CONVENIENCE SAMPLING]","[(0, 6), (116, 136)]"
5913,"surveys, observations, focus group interviews ...","[FOCUS GROUPS, SURVEY, INTERVIEW, OBSERVATION]","[(0, 6), (9, 20), (23, 34), (35, 44)]"
5914,the development of a web mapping system and it...,"[FORECASTING, MAPPING]","[(25, 32), (222, 233)]"
5916,through field observation and depth interview,"[INTERVIEW, OBSERVATION]","[(14, 25), (36, 45)]"


In [ ]:
entity_type = "method"

In [ ]:
df_f["Location_"] = df_f["Location"].apply(lambda x: [list(a) for a in x])

df_f["Location_"].apply(lambda x: [a.append(entity_type) for a in x])

df_f["Location_1"] = df_f["Location_"].apply(lambda x: [tuple(a) for a in x])

In [ ]:
df_f

,Description,Title,Location,Location_,Location_1
0,It explores how these practices can be conside...,[DISCOURSE],"[(82, 91)]","[[82, 91, method]]","[(82, 91, method)]"
1,Official Tourism Websites: A Discourse Analysi...,[DISCOURSE],"[(29, 38)]","[[29, 38, method]]","[(29, 38, method)]"
2,Based on ethnographic fieldwork among three no...,[DISCOURSE],"[(260, 269)]","[[260, 269, method]]","[(260, 269, method)]"
3,I argue that their testimonies of desire are s...,[DISCOURSE],"[(67, 76)]","[[67, 76, method]]","[(67, 76, method)]"
4,Drawing on recently developed frameworks for s...,[DISCOURSE],"[(58, 67)]","[[58, 67, method]]","[(58, 67, method)]"
...,...,...,...,...,...
5912,survey responses were collected from non-Malac...,"[SURVEY, CONVENIENCE SAMPLING]","[(0, 6), (116, 136)]","[[0, 6, method], [116, 136, method]]","[(0, 6, method), (116, 136, method)]"
5913,"surveys, observations, focus group interviews ...","[FOCUS GROUPS, SURVEY, INTERVIEW, OBSERVATION]","[(0, 6), (9, 20), (23, 34), (35, 44)]","[[0, 6, method], [9, 20, method], [23, 34, met...","[(0, 6, method), (9, 20, method), (23, 34, met..."
5914,the development of a web mapping system and it...,"[FORECASTING, MAPPING]","[(25, 32), (222, 233)]","[[25, 32, method], [222, 233, method]]","[(25, 32, method), (222, 233, method)]"
5916,through field observation and depth interview,"[INTERVIEW, OBSERVATION]","[(14, 25), (36, 45)]","[[14, 25, method], [36, 45, method]]","[(14, 25, method), (36, 45, method)]"


In [ ]:
# df_f.to_pickle("/content/drive/My Drive/Data/df_f.pkl")

In [ ]:
desc_list = list(df_f.Description)
location_list = list(df_f.Location_1)

In [ ]:
train_list=[]
for job, location in zip(desc_list, location_list):
  loc_dict={}
  loc_dict["entities"] = location
  temp_list = [job,loc_dict]
  temp_tuple= tuple(temp_list)
  train_list.append(temp_tuple)

In [ ]:
train_list[-30:]

[('Within the scope of the field study, three excursions were arranged to the town of Bozuyuk where the series was shot, and an on-site observation was made by witnessing both interviews and shooting of the series',
  {'entities': [(133, 144, 'method'), (173, 182, 'method')]}),
 ('Within this context, two candidate SARIMA models are evaluated for their forecasting ability using conventional diagnostic statistics for goodness of fit together with comparisons made with alternative establishment forecasts',
  {'entities': [(36, 41, 'method'), (73, 84, 'method')]}),
 ('Word frequency analysis, semantic network analysis, and sentiment analysis in big data text analysis are applied',
  {'entities': [(56, 73, 'method'), (87, 99, 'method')]}),
 ('Yet beyond the dominant discourses of literary and historical textual analysis, focused interpretation of the role of guidebooks in the tourism system is rare',
  {'entities': [(24, 33, 'method'), (62, 77, 'method')]}),
 ('and (c) To what extent does 

In [ ]:
df_tax_train.Description.iloc[-2]

'This study utilized Single Target Implicit Association Tests to bridge this gap by examining the influences of individuals dual attitudes toward leisure use of BSPs'

In [ ]:
tax_list

['TELEPHONE SURVEY',
 'DISCOURSE',
 'TIME-SERIES CROSS-SECTION',
 'SELF ORGANIZING MAPS',
 'QUOTA SAMPLING',
 'SCATTER PLOT',
 'TREND ANALYSIS',
 'DBNS',
 'POINTINTERP',
 'NARRATIVE INTERVIEWING',
 'SEMI-SUPERVISED LEARNING',
 'SERIAL CORRELATION',
 'COHORT STUDIES',
 'LONG SHORT TERM MEMORY NETWORKS',
 'TREE VIEWER',
 'CONTINUOUS',
 'HISTORICAL RESEARCH',
 'SOCIOMETRY',
 'FACTORIAL DESIGN',
 'CO-WORD ANALYSIS',
 'PAM',
 'DRL',
 'TOKENIZATION',
 'ELECTROENCEPHALOGRAPHY',
 'QUALITATIVE RESEARCH',
 'INTERVENTION ANALYSIS',
 'BUBBLE CLOUD',
 'LATENT BUDGET ANALYSIS',
 'PROGOL',
 'MULTIOUTPUT ALGORITHMS',
 'CONVERGENT VALIDITY',
 'MLPS',
 'NAMED ENTITY RECOGNITION',
 'INVERSE DISTANCE WEIGHTING',
 'SINTAX',
 'COREFERENCE RESOLUTION',
 'HISTORY ANALYSIS',
 'EXPERT INTERVIEWS',
 'PRE-EXPERIMENTAL RESEARCH',
 'GENERATIVE ADVERSARIAL NETWORKS',
 'HDBSCAN',
 'CYBERMETRICS',
 'SURVIVAL ANALYSIS',
 'FULL-PROFILE CONJOINT ANALYSIS',
 'CO-AUTHOR ANALYSIS',
 'MORPHOLOGICAL ANALYSIS',
 'PHENOMENOLOGY

In [ ]:
len(entity_dict)

In [ ]:
entity_dict

{'ABDUCTION': 5,
 'ABDUCTIVE': 6,
 'ACCIDENTAL SAMPLING': 24,
 'ACTION RESEARCH': 114,
 'ADAPTIVE CONJOINT ANALYSIS': 2,
 'ALTMETRICS': 5,
 'AMOS': 44,
 'ANALYTICAL RESEARCH': 8,
 'ANOMALY DETECTION': 3,
 'APPLIED RESEARCH': 69,
 'APRIORI ALGORITHM': 3,
 'ARCHIVAL RESEARCH': 36,
 'AREA SAMPLING': 1,
 'ARIMA': 196,
 'ATLAS.TI': 15,
 'AUTOBIOGRAPHY': 3,
 'AUTOENCODERS': 3,
 'AUTOETHNOGRAPHY': 22,
 'AXIAL CODING': 3,
 'BAR CHART': 1,
 'BAYESIAN STATISTICS': 2,
 'BIBLIOGRAPHIC ANALYSIS': 3,
 'BIBLIOGRAPHICAL COUPLING': 1,
 'BIBLIOMETRIC': 232,
 'BIOGRAPHY': 22,
 'BOOTSTRAPPING': 30,
 'CA': 26,
 'CAQDAS': 1,
 'CARTOGRAM': 5,
 'CASE STUDY': 3491,
 'CDA': 9,
 'CFA': 18,
 'CHOICE-BASED CONJOINT ANALYSIS': 2,
 'CITATION ANALYSIS': 41,
 'CLASSIFICATION': 928,
 'CLUSTERING': 369,
 'CNNS': 4,
 'CO-CITATION ANALYSIS': 22,
 'CO-WORD ANALYSIS': 12,
 'COHORT ANALYSIS': 10,
 'COHORT STUDIES': 1,
 'COLLABORATIVE RESEARCH': 30,
 'COMPUTATIONAL MODELING': 3,
 'COMPUTER-ASSISTED PERSONAL INTERVIEWING': 1,


In [ ]:
dct = {k:[v] for k,v in entity_dict.items()}
df_tr = pd.DataFrame(dct).T.sort_values(0, ascending=False).reset_index()
df_tr.columns = ["method", "freq"]
df_tr

,method,freq
0,SURVEY,7173
1,INTERVIEW,5794
2,CASE STUDY,3491
3,OBSERVATION,2047
4,DISCOURSE,1891
...,...,...
337,ECA,1
338,MANIFOLD LEARNING,1
339,CROSS-NATIONAL RESEARCH,1
340,DEEP BELIEF NETWORKS,1


In [ ]:
len([t for t in tax_list if t not in df_tr.method.tolist()])

163

In [ ]:
df_trx = pd.DataFrame({"method" : [t for t in tax_list if t not in df_tr.method.tolist()]})
df_trx


,method
0,TIME-SERIES CROSS-SECTION
1,SELF ORGANIZING MAPS
2,DBNS
3,POINTINTERP
4,NARRATIVE INTERVIEWING
...,...
158,QDA
159,RELATIONAL SEMANTICS
160,PIECEWISE CONSTANT INTERPOLATION
161,CORRELATIONAL RESEARCH


In [ ]:
#with open('/content/drive/My Drive/Data/tax_train_list_4.pkl', 'wb') as file:
#     pickle.dump(train_list,file)

In [ ]:
with open("/content/drive/My Drive/Data/tax_train_list_4.pkl", "rb") as fp:   # Unpickling
  train_data = pickle.load(fp)

In [ ]:
len(train_data)

32891

In [ ]:
# df_tr.to_excel("found_3.xlsx")

In [ ]:
# df_trx.to_excel("notfound.xlsx")

In [ ]:
train_data[:10000]

[('It explores how these practices can be considered more deeply through theoretical discourses of capitalism, identity construction, technology and social media, risk-taking, personal development, equalities, and sustainability',
  {'entities': [(82, 91, 'method')]}),
 ('Official Tourism Websites: A Discourse Analysis Perspective investigates the construction and promotion of identity of tourist locales by the designers of the official websites for destinations such as Santiago de Compostela, Spain; the Baltic states of Latvia and Estonia; New Orleans, Louisiana and Gary, Indiana; Myanmar/Burma; US Sports Halls of Fame; and, in recognizing the influence and popularity of such sites, three websites parodying the imaginary nations of Phaic Tan, Molvania, and San Sombrero',
  {'entities': [(29, 38, 'method')]}),
 ('Based on ethnographic fieldwork among three non-governmental organizations that use volunteer tourism as a social and economic development strategy as well as popular media te

In [ ]:
tr =  sorted(train_data, key= lambda x: x[0])

In [ ]:
df_tr = pd.DataFrame(tr)
df_tr

,0,1
0,A 1995 survey by the Travel Industry Associati...,"{'entities': [(7, 13, 'method')]}"
1,A 2 2 destination competitiveness matrix base...,"{'entities': [(35, 41, 'method')]}"
2,A 2 2 factorial experimental design was used ...,"{'entities': [(17, 27, 'method')]}"
3,A 2 2 quasi-experimental design is utilised t...,"{'entities': [(13, 23, 'method')]}"
4,A 2 (goal-framed messages: gain vs. loss) 2 (...,"{'entities': [(107, 117, 'method')]}"
...,...,...
32886,"wellness tourism, health tourism, medical tour...","{'entities': [(70, 84, 'method')]}"
32887,"were used for the comparative analysis, factor...","{'entities': [(92, 98, 'method')]}"
32888,what approaches have been utilised in mixed me...,"{'entities': [(38, 50, 'method')]}"
32889,which are adapted to fit to the tourism sector...,"{'entities': [(113, 127, 'method')]}"




```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```



In [ ]:
from spacy.util import minibatch, compounding

# nlp = spacy.load("en_core_web_sm")
labels = ["method"]

def train_model(TRAIN_DATA, n_iter=10):

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)

    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for label in labels:
      ner.add_label(label)
    
    optimizer = nlp.resume_training()
    move_names = list(ner.move_names) 

    # List of pipes you want to train
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

    # List of pipes which should remain unaffected in training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

    # only train NER
    with nlp.disable_pipes(*other_pipes):

      sizes = compounding(1.0, 4.0, 1.001)
      
      for itn in range(n_iter):
        print(f"starting iteration: {itn}")
        random.shuffle(TRAIN_DATA)
      
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=sizes)
        
        # dictionary to store losses
        losses = {}
        
        for batch in batches:
          texts, annotations = zip(*batch)

          # Calling update() over the iteration
          nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
        
        print("Losses", losses)     

In [ ]:
train_model(train_data, n_iter=15)

starting iteration: 0
Losses {'ner': 539587.8856728321}
starting iteration: 1
Losses {'ner': 526440.5126887338}
starting iteration: 2
Losses {'ner': 524204.3578190659}
starting iteration: 3
Losses {'ner': 522179.111576713}
starting iteration: 4
Losses {'ner': 520593.232169604}
starting iteration: 5
Losses {'ner': 519560.9857756442}
starting iteration: 6
Losses {'ner': 522830.9326835707}
starting iteration: 7
Losses {'ner': 519704.7195328498}
starting iteration: 8
Losses {'ner': 518140.14388523035}
starting iteration: 9
Losses {'ner': 520747.4195437782}
starting iteration: 10
Losses {'ner': 521291.2847159753}
starting iteration: 11
Losses {'ner': 519174.59940840735}
starting iteration: 12
Losses {'ner': 521581.82568801945}
starting iteration: 13
Losses {'ner': 521898.380214814}
starting iteration: 14
Losses {'ner': 521471.2998514711}


In [ ]:
nlp.to_disk("/content/drive/My Drive/Data/tax_method_model_4")

In [ ]:
nlp_model = spacy.load("/content/drive/My Drive/Data/tax_method_model_4")

In [ ]:
text = df.Abstract.iloc[1]
text

'Research on sustainability in tourism has increasingly focused on environmental and economic – rather than social – sustainability, especially in its practical applications. This case study of responsible tourism in Kumarakom, India, contributes to the growing body of literature on social sustainability, following a qualitative research design adopting both method and data triangulation. This includes field visits, interviews, and document analysis. The findings demonstrate the exceptional level of community participation and satisfaction at Kumarakom. This has been achieved through the continuous support, coordination, and collaboration of various stakeholders and strong, efficient government support. Moreover, the results highlight the significance of gender equality and women’s empowerment as integral elements of social sustainability. This study contributes to the development of tourism theory and governance by demonstrating how the substantive and procedural aspects of social sus

In [ ]:
doc = nlp_model(text)

In [ ]:
for ent in doc.ents:
  print("text: " + ent.text + " | " + "label: " + ent.label_)

text: case study | label: method
text: qualitative research | label: method
text: triangulation | label: method


In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [ ]:
for entity in tax_list:
  if re.search(entity, text, re.I):
    print([(m.start(), m.end(), entity) for m in re.finditer(entity, text, re.I)])

In [ ]:
train_data[0][0]

'By using Latent Dirichlet Allocation (LDA) topic modeling and state-of-the-art machine learning (ML) methodologies, we found that review readability in textual content and salient objects in images in visual content have a significant impact on both review usefulness and review enjoyment'

In [ ]:
n=0
for i in range(len(train_data)):
  text= train_data[i][0]
  doc = nlp_model(text)
  if len(doc.ents)<1:
    n+=1
  # for ent in doc.ents:
  #   print("text: " + ent.text + " | " + "label: " + ent.label_)
print(n)

6289


In [ ]:
len(train_data)

32891

In [ ]:
6289 / len(train_data)

0.19120732115168282

In [ ]:
26587 / 80878

0.3287296916343134

In [ ]:
25550/80427

0.31767938627575315

In [ ]:
text = df.methodology.iloc[121]
text

'Method\nMeasures for study variables\nMeasures for variables in this study were adopted from previous studies (Ajzen, 1991; Filieri and Mcleay, 2014; Meng and Choi, 2016; Lee and Tseng, 2015; Wheeler et al., 2005). The validity of the instruments used in our study were demonstrated in previous tourism and marketing studies. Specifically, three items were used for information authenticity. Three items were adopted to evaluate information sensation. Three items were used for star ratings. Three items were used to evaluate customer reviews. Finally, to evaluate involvement in the ELM, three items were adopted from prior studies. As for the constructs within TPB, three items were adopted to measure attitude; three items were used for subjective norms; three items were used to measure perceived behavior control; Finally, three items were used to estimate intention. All items used in our current study were measured by a Likert scale ranging from one to seven points (Table I). The items were

In [ ]:
doc = nlp_model(text)

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [ ]:
text="""
RESEARCH METHOD
Data Collection and Sample Description
The data were provided by Außenwirtschaft Österreich,
a branch of the Austrian Chamber of Commerce that
supports export efforts, and consisted of 194 exporting
firms that had completed an online questionnaire; the
respondents were all senior executives (e.g., chief executive
officers, export directors, export department heads)
with direct responsibility for export decisions. We
excluded 21 firms because of poorly completed questionnaires,
resulting in 173 fully usable responses. Of
the responding companies, 80% were independent companies
(either privately owned or public limited companies),
and 12% were part of a global company; the rest
had other corporate structures. Half the companies were
founded in the preceding 20 years, and their export
experience ranged from 1 to 125 years, with a median
of 12 (M = 20, SD = 21) years. On average, the firms
had 191 full-time employees (range from 1 to 7,800).
Almost all (95%) exported to the European Union, 64%
to other European countries, 38% to Asia, 36% to
Africa and/or the Middle East, 29% to North America,
20% to South or Central America, and 19% to Australia
and/or New Zealand. Half the companies
exported to 10 countries or fewer (M = 20, SD = 25),
and the average export dependence (i.e., export-to-total
sales ratio) was 53% (SD = 33%).
To judge the representativeness of our sample, we compared
it with official statistical data on exporting.
According to Statistik Austria, the national statistics
office, 43% of all Austrian exporters sell to a single
country only; however, the volume of exports accounted
for by this group is only 2% (Hodel 2004, 2012). This
group is of little relevance for the current study because
firms with a single export destination are unlikely to
engage in systematic segmentation activities (or to participate
in a study on export segmentation). This is
indeed reflected in the composition of our sample: only
2% of the respondent firms export to a single country
only. The majority of our respondents export to
between 2 and 19 countries (66%) and the rest (32%)
to 20+ countries. The corresponding groups in the
population are 52% and 6% and account for 30% and
68% of all exports, respectively (Hodel 2012). Thus,
our sample seems well representative of the Austrian
companies that are responsible for the bulk of the country’s
export activity and for which export segmentation
is likely to be important, considering the large number
of export markets served.4
Construct Measurement
Table 2 lists the measurements of all constructs in our
model, together with relevant psychometric information.
We measured firm size by the log-transformed number of
employees (Erramilli 1991), export experience by the
years a firm has been exporting (Winklhofer and Diamantopoulos
2002), export dependence as the exportto-
total sales ratio (Diamantopoulos and Inglis 1988;
Katsikeas 1994), export diversity by the number of
countries to which a firm exports (Dhanaraj and
Beamish 2003),5 and export market turbulence according
to Jaworski and Kohli’s (1993) well-established
scale.
For export segmentation commitment, we developed
our own five-item scale, which captured managers’ attitudes
and behavior toward the market segmentation
process. For export segmentation strategy, we itemized
the three strategic options and asked respondents to
specify their agreement with each option. This approach
acknowledges that the fit of the strategy with its context
is critical (Hultman, Robson, and Katsikeas 2009). We
specified segmentation bases as the number of distinct
segmentation variables the firm used (based on a checklist
of 21 criteria for segmenting business-to-consumer
markets and 14 criteria for segmenting business-tobusiness
markets).
For export segmentation effectiveness, we adapted the
basic four-dimensional structure developed by Foedermayr
and colleagues (Foedermayr and Diamantopoulos
2008a; Foedermayr, Diamantopoulos, and Sichtmann
2009), which consisted of targeting performance, positioning
performance, adaptability to change, and cost
reduction.6 Originally, the dimensions were operationalized
with formative indicators; however, the majority of
items under each dimension in the original model seem
to share a common cause, which suggests that reflective
measurement is more appropriate. The psychometric
properties shown in Table 2 strongly support this measurement
respecification.
For strategic export performance and financial export
performance, we adapted items from Zou and Cavusgil
(2002) and Zou, Taylor, and Osland (1998). The indicators
for strategic export performance are export market
share and rate of new market entry, and export financial
performance consists of export sales, export profits, and
export growth. Customer satisfaction comprises three
items about offerings, how consumers are treated, and
how their complaints have been handled. Consistent
with other investigations of export performance (e.g.,
Hooley et al. 2005; Hultman, Robson, and Katsikeas
2009; Morgan, Kaleka, and Katsikeas 2004), we relied
on self-reported measures.

"""

In [ ]:
doc = nlp_model(text)

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [ ]:
for entity in quantitative_entity_list:
    if re.search(entity, text, re.I):
      print([(m.start(), m.end(), entity) for m in re.finditer(entity, text, re.I)])

[(3119, 3124, 'scale'), (3193, 3198, 'scale')]
[(251, 264, 'questionnaire'), (483, 496, 'questionnaire')]
[(37, 43, 'sample'), (1399, 1405, 'sample'), (1952, 1958, 'sample'), (2276, 2282, 'sample')]
[(2597, 2602, 'model'), (4258, 4263, 'model')]


In [ ]:
df_z = pd.read_pickle("/content/drive/My Drive/Data/df_f.pkl")
df_z

,Description,Title,Location,Location_,Location_1
0,It explores how these practices can be conside...,[DISCOURSE],"[(82, 91)]","[[82, 91, method]]","[(82, 91, method)]"
1,Official Tourism Websites: A Discourse Analysi...,[DISCOURSE],"[(29, 38)]","[[29, 38, method]]","[(29, 38, method)]"
2,Based on ethnographic fieldwork among three no...,[DISCOURSE],"[(260, 269)]","[[260, 269, method]]","[(260, 269, method)]"
3,I argue that their testimonies of desire are s...,[DISCOURSE],"[(67, 76)]","[[67, 76, method]]","[(67, 76, method)]"
4,Drawing on recently developed frameworks for s...,[DISCOURSE],"[(58, 67)]","[[58, 67, method]]","[(58, 67, method)]"
...,...,...,...,...,...
5912,survey responses were collected from non-Malac...,"[SURVEY, CONVENIENCE SAMPLING]","[(0, 6), (116, 136)]","[[0, 6, method], [116, 136, method]]","[(0, 6, method), (116, 136, method)]"
5913,"surveys, observations, focus group interviews ...","[FOCUS GROUPS, SURVEY, INTERVIEW, OBSERVATION]","[(0, 6), (9, 20), (23, 34), (35, 44)]","[[0, 6, method], [9, 20, method], [23, 34, met...","[(0, 6, method), (9, 20, method), (23, 34, met..."
5914,the development of a web mapping system and it...,"[FORECASTING, MAPPING]","[(25, 32), (222, 233)]","[[25, 32, method], [222, 233, method]]","[(25, 32, method), (222, 233, method)]"
5916,through field observation and depth interview,"[INTERVIEW, OBSERVATION]","[(14, 25), (36, 45)]","[[14, 25, method], [36, 45, method]]","[(14, 25, method), (36, 45, method)]"


In [ ]:
found = list(df_z.Title.explode().unique())
len(found)

290

In [ ]:
len(tax_list)

505

In [ ]:
not_found = (set(tax_list).difference(set(found)))
len(not_found)


215

In [ ]:
pd.DataFrame(list(not_found)).to_excel("not_found.xlsx")